In [ ]:
# -*- coding: utf-8 -*-
"""
Created on 2023/04/11
Revised on 2023/05/30
 
@author: mjofre - Marc Jofre
e-mail: marc.jofre@upc.edu
Technical University of Catalonia - Universitat Politècnica de Catalunya (UPC)
"""
###########################################################################
# Seminar on Quantum Technologies for Cybersecurity: Networking and Systems
# Class
# Session 4 - Quantum instruments and infrastructure

import os, sys, time
import numpy as np
import math
import matplotlib.pyplot as plt
from google.colab import files
import matplotlib.style
import matplotlib as mpl
#print(plt.style.available)
mpl.style.use('default')

try:  
  import qiskit
except:
  print("installing qiskit...")
  !pip install qiskit --quiet
  print("installed qiskit.")
  import qiskit


installing qiskit...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 5.6 MB/s eta 0:00:00
installed qiskit.


In [ ]:
try:
  import sequence
  # https://github.com/sequence-toolbox/SeQUeNCe
except ImportError:
  print("installing sequence-toolbox...")
  !pip install git+https://github.com/sequence-toolbox/SeQUeNCe.git --quiet
  #!pip install sequence_toolbox --quiet
  print("installed sequence-toolbox.")
  import sequence

installing sequence-toolbox...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 6.6 MB/s eta 0:00:00
installed sequence-toolbox.


In [ ]:
# Analysis of the QKD performance of BB84 vs. devices performance
from ipywidgets import interact
from matplotlib import pyplot as plt
import time
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QKDNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel
from sequence.qkd.BB84 import pair_bb84_protocols
import sequence.utils.log as log

# constants
runtime = 2e8
sim_time = 100
####################################
tl = Timeline(runtime)
tl.show_progress = True

# set log
log_filename = "bb84.log"
log.set_logger(__name__, tl, log_filename)
log._log_modules=[] # Empty the list
log.set_logger_level('INFO') # 'DEBUG',  'INFO'
#log.track_module('BB84')
#log.track_module('timeline')
#log.track_module('light_source')

class KeyManager():
  def __init__(self, timeline, keysize, num_keys):
    self.timeline = timeline
    self.lower_protocols = []
    self.keysize = keysize
    self.num_keys = num_keys
    self.keys = []
    self.times = []
      
  def send_request(self):
    for p in self.lower_protocols:
      p.push(self.keysize, self.num_keys) # interface for BB84 to generate key
          
  def pop(self, info): # interface for BB84 to return generated keys
    self.keys.append(info)
    self.times.append(self.timeline.now() * 1e-9)

def test( keysize, distance, PolFidelity, attenuation, MeanPhotNumber, EffDetector, DarkCountDetector, TimeResDetector):
  """
  sim_time: duration of simulation time (ms)
  keysize: size of generated secure key (bits)
  """
  # begin by defining the simulation timeline with the correct simulation time
  tl = Timeline(sim_time * 1e9)

  # Alice
  ls_params = {"frequency": frequency, "mean_photon_num": MeanPhotNumber}
  n1 = QKDNode("n1", tl, stack_size=1)
  n1.set_seed(0)

  for name, param in ls_params.items():
    n1.update_lightsource_params(name, param)

  # Bob
  detector_params = [{"efficiency": EffDetector, "dark_count": DarkCountDetector, "time_resolution": TimeResDetector, "count_rate": detector_count_rate},
                    {"efficiency": EffDetector, "dark_count": DarkCountDetector, "time_resolution": TimeResDetector, "count_rate": detector_count_rate}]
  n2 = QKDNode("n2", tl, stack_size=1)
  n2.set_seed(1)

  for i in range(len(detector_params)):
    for name, param in detector_params[i].items():
      n2.update_detector_params(i, name, param)
  
  # Here, we create nodes for the network (QKD nodes for key distribution)
  # stack_size=1 indicates that only the BB84 protocol should be included
  
  pair_bb84_protocols(n1.protocol_stack[0], n2.protocol_stack[0])
  
  # connect the nodes and set parameters for the fibers
  # note that channels are one-way
  # construct a classical communication channel
  # (with arguments for the channel name, timeline, and length (in m))
  cc0 = ClassicalChannel("cc_n1_n2", tl, distance=distance)
  cc1 = ClassicalChannel("cc_n2_n1", tl, distance=distance)
  cc0.set_ends(n1, n2.name)
  cc1.set_ends(n2, n1.name)

  # construct a quantum communication channel
  # (with arguments for the channel name, timeline, attenuation (in dB/km), and distance (in m))
  qc0 = QuantumChannel("qc_n1_n2", tl, attenuation=attenuation, distance=distance,polarization_fidelity=PolFidelity)
  qc1 = QuantumChannel("qc_n2_n1", tl, attenuation=attenuation, distance=distance,polarization_fidelity=PolFidelity)
  qc0.set_ends(n1, n2.name)
  qc1.set_ends(n2, n1.name)
  
  # instantiate our written keysize protocol
  km1 = KeyManager(tl, keysize, 25)
  km1.lower_protocols.append(n1.protocol_stack[0])
  n1.protocol_stack[0].upper_protocols.append(km1)
  km2 = KeyManager(tl, keysize, 25)
  km2.lower_protocols.append(n2.protocol_stack[0])
  n2.protocol_stack[0].upper_protocols.append(km2)
  
  # start simulation and record timing
  tl.init()
  km1.send_request()
  tick = time.time()
  tl.run()
  print("execution time %.2f sec" % (time.time() - tick))
  plt.figure()
  # display our collected metrics
  line1,=plt.plot(km1.times, range(1, len(km1.keys) + 1), 'royalblue', marker="o")
  #print(km1.times)
  #print(km1.keys)
  
  #print(n1.protocol_stack[0].error_rates[0])
  #print("key error rates:")
   
  # Number of useful keys
  UsefullKeysAccumulatedCount=np.zeros_like(km1.times)
  for i, e in enumerate(n1.protocol_stack[0].error_rates):
    if (n1.protocol_stack[0].error_rates[i]<0.11):
      if (i==0):
        UsefullKeysAccumulatedCount[i]=1
      else:
        UsefullKeysAccumulatedCount[i]=1+UsefullKeysAccumulatedCount[i-1]
    else:
      UsefullKeysAccumulatedCount[i]=np.max(UsefullKeysAccumulatedCount)

  line2,=plt.plot(km1.times, UsefullKeysAccumulatedCount, 'forestgreen', marker="+")
  plt.legend((line1, line2), ('Completed', 'Sifted'), loc="best",shadow = False, fancybox = False, frameon = False, fontsize='small')# 'best'
  plt.xlabel("Simulation time (ms)")
  plt.ylabel("Number of Keys")
  plt.show()

  for i, e in enumerate(n1.protocol_stack[0].error_rates):
    print("\t key {}:\t{}%".format(i + 1, e * 100))

# Channel
#distance = 1e3
#PolFidelity=0.97 # Polarization fidelity

# Source
#attenuation=0.0002 # channel attenuation (in dB/km)
#MeanPhotNumber=0.1 # Photons per pulse
frequency=100e6 # Hz [pulse per second]

# Detectors:
#EffDetector=0.8 # Detector efficiency
#DarkCountDetector=10 # counts per second
#TimeResDetector=10 # nanoseconds
detector_count_rate=50e6 # maximum count rate detector


# Create and run the simulation
interactive_plot = interact(test,  keysize=[128, 256, 512], distance=[100, 1000, 10000], PolFidelity=(0.0,1.0,0.05), attenuation=[0.01,0.1,0.2], MeanPhotNumber=[0.05,0.1,0.5], EffDetector=(0.0,1.0,0.1),DarkCountDetector=[10,100,1000],TimeResDetector=[1,10,100])
interactive_plot

interactive(children=(Dropdown(description='keysize', options=(128, 256, 512), value=128), Dropdown(descriptio…

<function __main__.test(keysize, distance, PolFidelity, attenuation, MeanPhotNumber, EffDetector, DarkCountDetector, TimeResDetector)>